In [1]:
import cv2
import numpy as np

In [3]:
def canny(image):
    #Gray
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    #Blurr
    blurr = cv2.GaussianBlur(gray, (5,5), 0)
    #Edge Detection
    canny = cv2.Canny(blurr,50,100)
    return canny

def area_of_interest(image):
    height = image.shape[0]
    polygons = np.array([
        [(1100, height), (200,height), (550,250)]
    ])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def draw_lines(image, lines):
    image_frame = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line
            cv2.line(image_frame, (x1,y1), (x2,y2), (255,0,0), 10)
    return image_frame

def make_coordinates(image, line_parameters):
    # from slop and y intercept to coordinated of a line x and y (reverse of function polyfit )
    slope, y_intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1*(1/2))
    x1 = int((y1 - y_intercept)/slope)
    x2 = int((y2 - y_intercept)/slope)
    return np.array([x1, y1, x2, y2])
    
def average_slope(image, lines):
    # This function will taking avg of all the slopes and y intercepts and then send it 
    # to make_coordinates to convert them into lines cordinates and return line coordinates
    left_fit = []
    right_fit = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        # polyfit gives the m and c values in y= mx+c.. by giving x and y, 1 is the degree of polynomial 
        parameters = np.polyfit((x1,x2), (y1,y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
            
    left_fit_avg = np.average(left_fit, axis=0)
    #p = np.poly1d(left_fit_avg)
    right_fit_avg = np.average(right_fit, axis=0)
    left_line = make_coordinates(image, left_fit_avg)
    right_line = make_coordinates(image, right_fit_avg)
    return np.array([left_line, right_line])

# On Image for checking
#img = cv2.imread('road.png')
#Copy
#img_copy = np.copy(img)
#canny_image = canny(img_copy)

#interest_image = area_of_interest(canny_image)
#lines = cv2.HoughLinesP(interest_image, 2, np.pi/180, 100, np.array([]), minLineLength=5, maxLineGap=5)
#avg_lines = average_slope(img_copy, lines)

#lane_line = draw_lines(img_copy, avg_lines)

#combined_image = cv2.addWeighted(img_copy, 0.8, lane_line, 1, 1)

#cv2.imshow("Image", combined_image)
#cv2.waitKey(0)

# Video
cap = cv2.VideoCapture('test2.mp4')

while(cap.isOpened()):
    ret, frame = cap.read()
    canny_image = canny(frame)
    
    interest_image = area_of_interest(canny_image)
    lines = cv2.HoughLinesP(interest_image, 2, np.pi/180, 100, np.array([]), minLineLength=5, maxLineGap=5)
    avg_lines = average_slope(frame, lines)
    lane_line = draw_lines(frame, avg_lines)
    combined_image = cv2.addWeighted(frame, 0.8, lane_line, 1, 1)

    cv2.imshow("Image", combined_image)
    if cv2.waitKey(10) == ord('x'):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\Aneeba\Anaconda3\lib\site-packages\numpy\lib\function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\Aneeba\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


TypeError: cannot unpack non-iterable numpy.float64 object